In [2]:
#In this code block implement our kaggle.json api key to use the dataset
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"cakmak61","key":"55d16274829171b5e05a5792176e1508"}'}

In [3]:
!pip install -q kaggle

# Here we move our api key to the correct folder
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Then Download Stanford Cars Dataset
!kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder

# Unzip the dataset
!unzip -q stanford-car-dataset-by-classes-folder.zip -d data/


Dataset URL: https://www.kaggle.com/datasets/jutrera/stanford-car-dataset-by-classes-folder
License(s): other
100% 1.82G/1.83G [00:08<00:00, 320MB/s]
100% 1.83G/1.83G [00:08<00:00, 244MB/s]


In [ ]:
# Here we import the related libraries
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm

# WE set device settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Our Hyperparameters
batch_size = 64
image_size = 64
channels = 3
latent_dim = 128
n_epochs = 200
lambda_gp = 10
n_critic = 5
lr = 1e-4
beta1 = 0.0
beta2 = 0.9

# Data transformations
transform = transforms.Compose([
    transforms.Resize(64),  # veya 128
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

# Uploading the dataset
data_root = './data/stanford_cars'
train_dataset = datasets.ImageFolder(root='data/car_data/car_data/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

# Generator Definition
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels):
        super(Generator, self).__init__()
        self.init_size = image_size // 16
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 512 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(512),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 256, 3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, img_channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 512, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

# Definition of Discriminator (Critic)
class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters))  # Instead of layer norm, we have utilized BatchNorm
            block.append(nn.LeakyReLU(0.2, inplace=True))
            return block

        self.model = nn.Sequential(
            *discriminator_block(img_channels, 64, bn=False),  # 64x64 -> 32x32
            *discriminator_block(64, 128),                      # 32x32 -> 16x16
            *discriminator_block(128, 256),                     # 16x16 -> 8x8
            *discriminator_block(256, 512),                     # 8x8 -> 4x4
        )

        ds_size = image_size // 2 ** 4  # 64 -> 4
        self.adv_layer = nn.Linear(512 * ds_size ** 2, 1)

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        return validity


# Calculating Gradient penalty
def compute_gradient_penalty(D, real_samples, fake_samples):
    alpha = torch.rand(real_samples.size(0), 1, 1, 1, device=device)
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = torch.ones(real_samples.size(0), 1, device=device, requires_grad=False)
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

# Model and Optimization
generator = Generator(latent_dim, channels).to(device)
discriminator = Discriminator(channels).to(device)

optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(beta1, beta2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, beta2))

# Training Loop
os.makedirs("wgan_gp_outputs", exist_ok=True)

for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(tqdm(train_loader)):
        real_imgs = imgs.to(device)

        #  Training Discriminator
        for _ in range(n_critic):
            z = torch.randn(imgs.size(0), latent_dim, device=device)
            fake_imgs = generator(z).detach()
            real_validity = discriminator(real_imgs)
            fake_validity = discriminator(fake_imgs)
            gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
            d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

            optimizer_D.zero_grad()
            d_loss.backward()
            optimizer_D.step()

        # Training Generator
        z = torch.randn(imgs.size(0), latent_dim, device=device)
        gen_imgs = generator(z)
        g_loss = -torch.mean(discriminator(gen_imgs))

        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()

    print(f"[Epoch {epoch+1}/{n_epochs}] D loss: {d_loss.item():.4f} | G loss: {g_loss.item():.4f}")

    # Recording the sample images
    if (epoch + 1) % 10 == 0 or epoch == 0:
        generator.eval()
        with torch.no_grad():
            sample_z = torch.randn(4, latent_dim, device=device)
            samples = generator(sample_z)
            grid = make_grid(samples.cpu(), nrow=2, normalize=True)
            save_image(grid, f"wgan_gp_outputs/sample_epoch_{epoch+1:03d}.png")
        generator.train()

print("Training is completed.")


  0%|          | 0/128 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:180.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 128/128 [00:27<00:00,  4.59it/s]


[Epoch 1/200] D loss: -16.1672 | G loss: 26.8570


100%|██████████| 128/128 [00:26<00:00,  4.77it/s]


[Epoch 2/200] D loss: -32.3738 | G loss: 30.1733


100%|██████████| 128/128 [00:26<00:00,  4.75it/s]


...

100%|██████████| 128/128 [00:27<00:00,  4.64it/s]


[Epoch 73/200] D loss: 13609.1035 | G loss: 34217.0273


100%|██████████| 128/128 [00:27<00:00,  4.69it/s]


[Epoch 74/200] D loss: 31634.8457 | G loss: 32979.0938


 79%|███████▉  | 101/128 [00:21<00:04,  5.64it/s]